In [ ]:
!pip install qiskit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# !pip install pylatexenc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# !apt install pylatexenc

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package pylatexenc


In [ ]:
from qiskit import QuantumCircuit, execute, Aer
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from numpy.random import randint
import numpy as np
# from qiskit.providers.aer import QasmSimulator
import pylatexenc
# from pylatexenc import MatplotlibDrawer

# import pylatexenc.MatplotlibDrawer as MatplotlibDrawer



In [ ]:
def bb84_circuit(state, basis, measurement_basis):
   
    #state: array of 0s and 1s denoting the state to be encoded
    #basis: array of 0s and 1s denoting the basis to be used for encoding
                #0 -> Computational Basis
                #1 -> Hadamard Basis
    #meas_basis: array of 0s and 1s denoting the basis to be used for measurement
                #0 -> Computational Basis
                #1 -> Hadamard Basis

    num_qubits = len(state)
    
    circuit = QuantumCircuit(num_qubits)

    # Sender prepares qubits
    for i in range(len(basis)):
        if state[i] == 1:
            circuit.x(i)
        if basis[i] == 1:
            circuit.h(i)
   

    # Measuring action performed by Bob
    for i in range(len(measurement_basis)):
        if measurement_basis[i] == 1:
            circuit.h(i)

       
    circuit.measure_all()
    # circuit.draw('mpl')
    
    return circuit

In [ ]:
num_qubits = 32

alice_basis = np.random.randint(2, size=num_qubits)
alice_state = np.random.randint(2, size=num_qubits)
bob_basis = np.random.randint(2, size=num_qubits)


print(f"Alice's State:\t {np.array2string(alice_state)}")
print(f"Alice's Bases:\t {np.array2string(alice_basis)}")
print(f"Bob's Bases:\t {np.array2string(bob_basis)}")

Alice's State:	 [0 1 1 1 0 0 1 1 1 0 0 0 0 1 0 1 0 0 1 1 0 0 0 0 0 0 1 1 0 1 0 0]
Alice's Bases:	 [0 1 0 1 0 0 1 1 0 0 1 1 1 1 1 1 0 1 0 0 1 0 1 0 0 1 0 1 0 1 0 0]
Bob's Bases:	 [0 0 1 1 0 1 1 0 1 0 0 0 1 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1]


In [ ]:
circuit = bb84_circuit(alice_state, alice_basis, bob_basis)
key = execute(circuit.reverse_bits(),backend=QasmSimulator(),shots=1).result().get_counts().most_frequent()

shifted_key = ''
for i in range(num_qubits):
    if alice_basis[i] == bob_basis[i]:
         shifted_key += str(key[i])
print(f"Key: {shifted_key} \nKey Size: {len(shifted_key)}")

Key: 01010001100100 
Key Size: 14


In [ ]:
from qiskit import QuantumCircuit, execute, Aer
from qiskit.providers.aer import QasmSimulator
import numpy as np

# num_qubits = 32

# alice_basis = np.random.randint(2, size=num_qubits)
# alice_state = np.random.randint(2, size=num_qubits)
# bob_basis = np.random.randint(2, size=num_qubits)

# print(f"Alice's State:\t {np.array2string(alice_state)}")
# print(f"Alice's Bases:\t {np.array2string(alice_basis)}")
# print(f"Bob's Bases:\t {np.array2string(bob_basis)}")

def bb84_circuit_eve(state, basis, measurement_basis, eve_present):
    num_qubits = len(state)
    circuit = QuantumCircuit(num_qubits, num_qubits)

    # Sender prepares qubits
    for i in range(num_qubits):
        if state[i] == 1:
            circuit.x(i)
        if basis[i] == 1:
            circuit.h(i)

    # Eavesdropper (Eve) intercepts and measures qubits
    if eve_present:
        eve_basis = np.random.randint(2, size=num_qubits)
        for i in range(num_qubits):
            if eve_basis[i] == 1:
                circuit.h(i)
            circuit.measure(i, i)
        # return circuit

    # Measuring action performed by Bob
    for i in range(num_qubits):
        if measurement_basis[i] == 1:
            circuit.h(i)
    
    circuit.measure(range(num_qubits), range(num_qubits))
    
    return circuit

In [ ]:
num_qubits = 32

alice_basis = np.random.randint(2, size=num_qubits)
alice_state = np.random.randint(2, size=num_qubits)
bob_basis = np.random.randint(2, size=num_qubits)


print(f"Alice's State:\t {np.array2string(alice_state)}")
print(f"Alice's Bases:\t {np.array2string(alice_basis)}")
print(f"Bob's Bases:\t {np.array2string(bob_basis)}")

Alice's State:	 [1 0 1 0 1 1 1 1 0 1 1 1 0 0 1 0 0 0 0 1 1 1 0 1 1 0 1 0 1 1 1 1]
Alice's Bases:	 [0 0 1 1 0 0 1 1 0 1 0 0 0 0 1 0 0 1 0 1 0 0 0 1 0 0 0 1 1 1 1 1]
Bob's Bases:	 [1 1 1 0 0 1 0 1 0 1 1 1 1 1 0 0 1 0 1 1 1 1 1 1 0 0 1 1 1 0 1 1]


In [ ]:
circuit = bb84_circuit_eve(alice_state, alice_basis, bob_basis, 1)
key = execute(circuit.reverse_bits(),backend=QasmSimulator(),shots=1).result().get_counts().most_frequent()

shifted_key = ''
bob_key = ''
count = 0
for i in range(num_qubits):
    if alice_basis[i] == bob_basis[i]:
         shifted_key += str(key[i])
         bob_key += str(alice_state[i])
         if(str(key[i]) != str(alice_state[i])):
            count += 1

print(f"Key: {shifted_key} \nBob's Key: {bob_key}")
print(f"Key Size: {len(shifted_key)}")

# count = 0
# for i in range(len(shifted_key)):
#     if shifted_key[i] != bob_key[i]:
#         count += 1
print(f"Difference between the shifted key and bob's key", len(shifted_key) - count)

Key: 10011001100000 
Bob's Key: 11101011100111
Key Size: 14
Difference between the shifted key and bob's key 7
